# Minimal NeRF
This is a simplied version of the method presented in *NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis*

1. [Project Website](http://www.matthewtancik.com/nerf)
1. [arXiv Paper](https://arxiv.org/abs/2003.08934)
1. [Full Code](github.com/bmild/nerf)

Components not included in the notebook
1. 5D input including view directions
1. Hierarchical Sampling
1. This is a synthetic dataset, for real images the camera positions can be estimated using [COLMAP](https://colmap.github.io/)



## Load libraries

In [ ]:
import os, sys
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
if not os.path.exists('tiny_nerf_data.npz'):
    !wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz

## Load Input Images and Poses

In [ ]:
data = np.load('tiny_nerf_data.npz')
images = data['images']   # Set of images
poses = data['poses']     # Camera positions in world coordinates
focal = data['focal']     # Focal length
H, W = images.shape[1:3]  # Image size


print("Images count: ", images.shape[0])
print("Images sizes: ",images.shape[-3:], "Metrix Ext", poses.shape[-2:], "Focal lenght: ", focal)

id_test = 101 # Integer between [101, 105]
testimg, testpose = images[id_test], poses[id_test]
images = images[:100,...,:3]
poses = poses[:100]

plt.imshow(testimg)
plt.show()

## Main functions

### Positional encoder

In [ ]:
def posenc(x):
  """Encodes the position into its corresponding Fourier feature.
    Args:
        x: The input coordinate.
    Returns:
        Fourier features tensors of the position.
  """
  rets = [x]
  for i in range(L_embed):
    for fn in [tf.sin, tf.cos]:
      rets.append(fn(2.**i * x))
  return tf.concat(rets, -1)

# Global variables
L_embed = 6
embed_fn = posenc

### MLP-NeRF


In [ ]:
def init_model(D=8, W=256):
    """Generates the NeRF neural network.

    Args:
      D: The number of MLP layers.
      W: The number of neurons of every channel.

    Returns:
        The `tf.keras` model.
    """
    relu = tf.keras.layers.ReLU()
    dense = lambda W=W, act=relu : tf.keras.layers.Dense(W, activation=act)

    inputs = tf.keras.Input(shape=(3 + 3*2*L_embed))
    outputs = inputs
    for i in range(D):
        outputs = dense()(outputs)
        if i%4==0 and i>0:
            outputs = tf.concat([outputs, inputs], -1)
    outputs = dense(4, act=None)(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

### Create rays

In [ ]:
def get_rays(H, W, focal, c2w):
    """Computes origin point and direction vector of rays.
    Args:
      H: Height of the image.
      W: Width of the image.
      focal: The focal length between the images and the camera.
      c2w: The pose matrix of the camera.

      Returns:
          Tuple of origin point and direction vector for rays.
    """
    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
    dirs = tf.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -tf.ones_like(i)], -1)
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
    rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
    return rays_o, rays_d

### Rendering

In [ ]:
def render_rays(network_fn, rays_o, rays_d, near, far, N_samples, rand=False):
    """Generates the RGB image and depth map from model prediction.

      Args:
          network_fn: The MLP model that is trained to predict the rgb and volume density of the volumetric scene.
          rays_o, rays_d: origin point and direction vector for rays.
          near: The near bound of the volumetric scene.
          far: The far bound of the volumetric scene.
          N_samples: Number of sample points in a ray.
          rand: Choice to randomise the sampling strategy.

      Returns:
          Tuple of rgb image and depth map, accuracy map.
    """
    def batchify(fn, chunk=1024*32):
        return lambda inputs : tf.concat([fn(inputs[i:i+chunk]) for i in range(0, inputs.shape[0], chunk)], 0)

    # Compute 3D query points
    z_vals = tf.linspace(near, far, N_samples)
    if rand:
      z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
    pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]

    # Run network
    pts_flat = tf.reshape(pts, [-1,3])
    pts_flat = embed_fn(pts_flat)
    raw = batchify(network_fn)(pts_flat)
    raw = tf.reshape(raw, list(pts.shape[:-1]) + [4])

    # Compute opacities and colors
    sigma_a = tf.nn.relu(raw[...,3])
    rgb = tf.math.sigmoid(raw[...,:3])

    # Do volume rendering
    dists = tf.concat([z_vals[..., 1:] - z_vals[..., :-1], tf.broadcast_to([1e10], z_vals[...,:1].shape)], -1)
    alpha = 1.-tf.exp(-sigma_a * dists)
    weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, -1, exclusive=True)

    rgb_map = tf.reduce_sum(weights[...,None] * rgb, -2)
    depth_map = tf.reduce_sum(weights * z_vals, -1)
    acc_map = tf.reduce_sum(weights, -1)

    return rgb_map, depth_map, acc_map

## Optimize NeRF

Here we optimize the model. We plot a rendered holdout view and its PSNR every 50 iterations.

In [ ]:
model = init_model()
optimizer = tf.keras.optimizers.Adam(5e-4)

# Parameters
N_samples = 64
N_iters   = 1000
Near      = 2.0
Far       = 6.0
Randomise = True
lr        = 5.0e-4


psnrs     = []
iternums  = []
i_plot    = 25


model = init_model()
optimizer = tf.keras.optimizers.Adam(lr)


import time
t = time.time()
for i in range(N_iters+1):

    img_i = np.random.randint(images.shape[0])
    target = images[img_i]
    pose = poses[img_i]
    rays_o, rays_d = get_rays(H, W, focal, pose)
    with tf.GradientTape() as tape:
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=Near, far=Far, N_samples=N_samples, rand=Randomise)
        loss = tf.reduce_mean(tf.square(rgb - target))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if i%i_plot==0:
        print(i, (time.time() - t) / i_plot, 'secs per iter')
        t = time.time()

        # Render the holdout view for logging
        rays_o, rays_d = get_rays(H, W, focal, testpose)
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
        loss = tf.reduce_mean(tf.square(rgb - testimg))
        psnr = -10. * tf.math.log(loss) / tf.math.log(10.)

        psnrs.append(psnr.numpy())
        iternums.append(i)

        plt.figure(figsize=(15,4))
        plt.subplot(131)
        plt.imshow(rgb)
        plt.title(f'Iteration: {i}')

        plt.subplot(132)
        plt.imshow(depth)
        plt.title(f'Depth map: {i}')


        plt.subplot(133)
        plt.plot(iternums, psnrs)
        plt.title('PSNR')
        plt.show()

print('Done')

## Interactive Visualization

In [ ]:
%matplotlib inline
from ipywidgets import interactive, widgets


trans_t = lambda t : tf.convert_to_tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=tf.float32)

rot_phi = lambda phi : tf.convert_to_tensor([
    [1,0,0,0],
    [0,tf.cos(phi),-tf.sin(phi),0],
    [0,tf.sin(phi), tf.cos(phi),0],
    [0,0,0,1],
], dtype=tf.float32)

rot_theta = lambda th : tf.convert_to_tensor([
    [tf.cos(th),0,-tf.sin(th),0],
    [0,1,0,0],
    [tf.sin(th),0, tf.cos(th),0],
    [0,0,0,1],
], dtype=tf.float32)


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi/180.*np.pi) @ c2w
    c2w = rot_theta(theta/180.*np.pi) @ c2w
    c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w


def f(**kwargs):
    c2w = pose_spherical(**kwargs)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
    img = np.clip(rgb,0,1)

    plt.figure(2, figsize=(20,6))
    plt.imshow(img)
    plt.show()


sldr = lambda v, mi, ma: widgets.FloatSlider(
    value=v,
    min=mi,
    max=ma,
    step=.01,
)

names = [
    ['theta', [100., 0., 360]],
    ['phi', [-30., -90, 0]],
    ['radius', [4., 3., 5.]],
]

interactive_plot = interactive(f, **{s[0] : sldr(*s[1]) for s in names})
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

## Render 360 Video

In [ ]:
frames = []
for th in tqdm(np.linspace(0., 360., 120, endpoint=False)):
    c2w = pose_spherical(th, -30., 4.)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb, depth, acc = render_rays(model, rays_o, rays_d, near=Near, far=Far, N_samples=N_samples)
    frames.append((255*np.clip(rgb,0,1)).astype(np.uint8))

import imageio
f = 'video.mp4'
imageio.mimwrite(f, frames, fps=30, quality=7)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)